<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/01_preprocessing/FF%2B%2B_download_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 버전을 명시하지 않고 최신 버전으로 설치
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from google.colab import drive
import os
import cv2
import json
import urllib.request
import tempfile
import random
from tqdm import tqdm
import urllib.request

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# 랜덤 시드 고정
SEED = 42
random.seed(SEED)

# --- 설정 ---
GDRIVE_BASE_PATH = "/content/drive/MyDrive/HECTO/Dataset/FF_frames"
NUM_FRAMES = 15
SERVER_URL = "http://kaldir.vc.in.tum.de/faceforensics/v3/"
FILELIST_URL = SERVER_URL + "misc/filelist.json"

# 조작 방식 리스트 (4가지 주요 방식)
MANIPULATIONS = ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']
COMPRESSION = 'c23'

In [ ]:
def get_balanced_dataset(sample_per_category=250, is_test=False):
    print("서버에서 파일 리스트를 가져오는 중...")
    with urllib.request.urlopen(FILELIST_URL) as url:
        file_pairs = json.loads(url.read().decode())

    # 테스트 모드인 경우 카테고리당 2개만 선택
    if is_test:
        sample_per_category = 2

    # 1. Real 영상 리스트 확보 (전체에서 샘플링)
    # 가짜가 총 4종 * 250 = 1000개이므로, 진짜도 1000개 전체 사용
    # 수정된 리스트 생성부 (셀 3 확인)
    real_list = []
    for pair in file_pairs:
        real_list.extend([pair[0] + '.mp4', pair[1] + '.mp4'])
    # 중복 제거 (혹시 모를 겹침 방지)
    real_list = list(set(real_list))

    if is_test:
        real_list = random.sample(real_list, 2)
    else:
        # 가짜 총합에 맞춰 1000개 샘플링 (FF++ 원본은 딱 1000개이므로 셔플만)
        random.shuffle(real_list)

    # 2. 가짜 영상 리스트 확보 (카테고리별 250개씩)
    fake_dict = {}
    for method in MANIPULATIONS:
        # 각 조작 방식별로 랜덤하게 pair 선택
        sampled_pairs = random.sample(file_pairs, sample_per_category)
        fake_dict[method] = ['_'.join(p) + '.mp4' for p in sampled_pairs]

    return real_list, fake_dict


In [ ]:
model_path = 'detector.tflite'
url = "https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite"
urllib.request.urlretrieve(url, model_path)

# 2. 검출기 설정
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

def crop_and_align_face_task(frame, detector, target_size=(256, 256)):
    # MediaPipe용 이미지 객체 생성
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 얼굴 검출
    detection_result = detector.detect(mp_image)

    if not detection_result.detections:
        return None

    # 첫 번째 얼굴 정보 가져오기
    detection = detection_result.detections[0]
    bbox = detection.bounding_box # 직접 픽셀 좌표가 나옵니다

    # 마진 계산 (25%)
    margin_w = int(bbox.width * 0.25)
    margin_h = int(bbox.height * 0.25)

    x1 = max(0, bbox.origin_x - margin_w)
    y1 = max(0, bbox.origin_y - margin_h)
    x2 = min(frame.shape[1], bbox.origin_x + bbox.width + margin_w)
    y2 = min(frame.shape[0], bbox.origin_y + bbox.height + margin_h)

    face_img = frame[y1:y2, x1:x2]
    if face_img.size == 0: return None

    return cv2.resize(face_img, target_size)

In [ ]:
def process_and_extract(video_names, sub_path, category_name, label, detector):
    """수정된 추출 메서드"""
    save_dir = os.path.join(GDRIVE_BASE_PATH, category_name)
    os.makedirs(save_dir, exist_ok=True)

    for v_name in tqdm(video_names, desc=f"Processing {category_name}"):
        video_url = SERVER_URL + sub_path + v_name
        video_id = v_name.split('.')[0]
        video_folder = os.path.join(save_dir, video_id)

        # 체크포인트: 이미 15프레임이 있으면 스킵
        if os.path.exists(video_folder) and len(os.listdir(video_folder)) >= NUM_FRAMES:
            continue

        os.makedirs(video_folder, exist_ok=True)

        with tempfile.NamedTemporaryFile(suffix='.mp4') as temp_v:
            try:
                urllib.request.urlretrieve(video_url, temp_v.name)
                cap = cv2.VideoCapture(temp_v.name)
                total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if total <= 0: continue

                interval = total // NUM_FRAMES
                success_count = 0

                for i in range(NUM_FRAMES):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
                    ret, frame = cap.read()
                    if ret:
                        # 얼굴 크롭 함수 호출 (detector 전달)
                        face = crop_and_align_face_task(frame, detector)
                        if face is not None:
                            img_name = f"f{i:03d}_{label}.jpg"
                            cv2.imwrite(os.path.join(video_folder, img_name), face)
                            success_count += 1
                cap.release()

                # 얼굴을 하나도 못 찾았다면 빈 폴더 삭제
                if success_count == 0:
                    os.rmdir(video_folder)
            except Exception as e:
                print(f"❌ Error {v_name}: {e}")

In [ ]:
# --- 실행 부분 ---
# 테스트 모드로 실행 (영상 각 2개씩) - 처음엔 True로 해서 잘 되는지 확인하는 걸 추천드려요!
real_videos, fake_videos_dict = get_balanced_dataset(is_test=False)
print(f"확인된 Real 영상 개수: {len(real_videos)}")

# 1. Real 영상 처리 (detector 인자 추가)
real_sub_path = f"original_sequences/youtube/{COMPRESSION}/videos/"
process_and_extract(real_videos, real_sub_path, "Real", 0, detector)

# 2. Fake 영상 카테고리별 처리 (detector 인자 추가)
for method, v_list in fake_videos_dict.items():
    fake_sub_path = f"manipulated_sequences/{method}/{COMPRESSION}/videos/"
    # 카테고리 이름 앞에 "Fake/"를 붙여서 폴더 구조 유지
    process_and_extract(v_list, fake_sub_path, f"Fake/{method}", 1, detector)

print("\n🎉 모든 처리가 완료되었습니다. 구글 드라이브에서 이미지를 확인해보세요!")

서버에서 파일 리스트를 가져오는 중...
확인된 Real 영상 개수: 1000


Processing Fake/NeuralTextures: 100%|██████████| 250/250 [38:48<00:00,  9.31s/it]


🎉 모든 처리가 완료되었습니다. 구글 드라이브에서 이미지를 확인해보세요!
